In [ ]:
#!/usr/bin/env python3
"""
balance_ledger.py

Automate ledger balancing.

Features
--------
* Reads a CSV with columns: Date, Description, Debit, Credit, [Balance]
* Supports optional starting balance.
* Computes running balance.
* Reports mismatches (if a Balance column is present).
* Writes a clean, balanced ledger to `balanced_ledger.csv`.
"""

import argparse
import csv
import sys
from decimal import Decimal, ROUND_HALF_UP
from pathlib import Path

# ----------------------------------------------------------------------
# Helper functions
# ----------------------------------------------------------------------
def parse_amount(s: str) -> Decimal:
    """Convert a string (e.g. '1,234.56' or '-567.00') to Decimal."""
    if not s:
        return Decimal('0')
    # Remove commas and any whitespace
    cleaned = s.replace(',', '').strip()
    return Decimal(cleaned).quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)


def read_ledger(csv_path: Path, start_balance: Decimal = Decimal('0')) -> list[dict]:
    """
    Return a list of transaction dicts with a computed running balance.
    """
    rows = []
    running = start_balance

    with csv_path.open(newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        expected_fields = {'Date', 'Description', 'Debit', 'Credit'}
        missing = expected_fields - set(reader.fieldnames or [])
        if missing:
            raise ValueError(f"CSV missing required columns: {', '.join(missing)}")

        for i, row in enumerate(reader, start=2):  # start=2 → line number in file
            try:
                debit = parse_amount(row.get('Debit', ''))
                credit = parse_amount(row.get('Credit', ''))
            except Exception as e:
                print(f"Line {i}: Invalid amount – {e}", file=sys.stderr)
                continue

            net = credit - debit
            running += net

            record = {
                'line': i,
                'date': row['Date'].strip(),
                'description': row['Description'].strip(),
                'debit': debit,
                'credit': credit,
                'net': net,
                'balance': running,
                'expected_balance': None,
            }

            # Optional: user-provided balance column
            if 'Balance' in row:
                try:
                    record['expected_balance'] = parse_amount(row['Balance'])
                except Exception:
                    print(f"Line {i}: Invalid Balance value", file=sys.stderr)

            rows.append(record)

    return rows


def write_balanced(csv_path: Path, records: list[dict]):
    """Write a clean CSV with computed balances."""
    fieldnames = [
        'Date', 'Description', 'Debit', 'Credit', 'Balance'
    ]
    with csv_path.open('w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for r in records:
            writer.writerow({
                'Date': r['date'],
                'Description': r['description'],
                'Debit': f"{r['debit']:,}",
                'Credit': f"{r['credit']:,}",
                'Balance': f"{r['balance']:,}",
            })


def report_mismatches(records: list[dict]):
    """Print any lines where computed != expected balance."""
    mismatches = [
        r for r in records
        if r['expected_balance'] is not None and r['balance'] != r['expected_balance']
    ]
    if not mismatches:
        print("All balances match!")
        return

    print("\nMISMATCHES FOUND:")
    print("-" * 80)
    print(f"{'Line':<6} {'Date':<12} {'Description':<30} {'Computed':<15} {'Expected':<15} {'Diff':<10}")
    print("-" * 80)
    for r in mismatches:
        diff = r['balance'] - r['expected_balance']
        print(f"{r['line']:<6} {r['date']:<12} {r['description']:<30} "
              f"{r['balance']:<15,} {r['expected_balance']:<15,} {diff:<+10,}")
    print("-" * 80)


# ----------------------------------------------------------------------
# CLI
# ----------------------------------------------------------------------
def main():
    parser = argparse.ArgumentParser(
        description="Automate ledger balancing from a CSV file."
    )
    parser.add_argument(
        "csv_file",
        type=Path,
        help="Path to the input ledger CSV."
    )
    parser.add_argument(
        "--start", "-s",
        type=Decimal,
        default=Decimal('0'),
        help="Starting balance (default: 0). Use commas if needed, e.g. 12,345.67"
    )
    parser.add_argument(
        "--output", "-o",
        type=Path,
        default=Path("balanced_ledger.csv"),
        help="Output CSV with computed balances (default: balanced_ledger.csv)"
    )
    args = parser.parse_args()

    if not args.csv_file.is_file():
        print(f"Error: File not found – {args.csv_file}", file=sys.stderr)
        sys.exit(1)

    try:
        records = read_ledger(args.csv_file, args.start)
    except Exception as e:
        print(f"Error reading ledger: {e}", file=sys.stderr)
        sys.exit(1)

    # Final balance
    final_balance = records[-1]['balance'] if records else args.start
    print(f"\nLedger processed – {len(records)} transactions")
    print(f"Final computed balance: {final_balance:,.2f}")

    # Report mismatches
    report_mismatches(records)

    # Write clean output
    write_balanced(args.output, records)
    print(f"\nBalanced ledger written to: {args.output.resolve()}")


if __name__ == "__main__":
    main()